In [1]:
#Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from dotenv import load_dotenv
import os

✅ Checklist de Exploração

1. **Visão Geral**

In [2]:
#Criando a conexao com a planilha do google
try:
    load_dotenv()  # Carrega as variáveis do .env
    sheet_id = os.getenv('SHEET_ID')
    sheet_name = os.getenv('SHEET_NAME')
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    print('Conectado ao Google Sheets com sucesso.')
except Exception as e:
    print(f"Erro ao conectar com o Google Sheets: {e}")


Conectado ao Google Sheets com sucesso.


In [3]:
#Transformando os dados em um DataFrame
df_dados = pd.read_csv(url)

In [4]:
df_dados.head().style.set_caption("Primeiras 5 linhas do dataset")

,Data,Tipo Receita/Despesa,Valor
0,01/01/25,Receita,"R$ 150,00"
1,02/01/25,Receita,"R$ 120,00"
2,02/01/25,Despesa Moto,"R$ 30,00"
3,02/01/25,Despesa Combustivel,"R$ 25,00"
4,02/01/25,Despesa Remedio,"R$ 65,00"


In [6]:
df_dados.sample(5).style.set_caption("5 linhas de amostras aleatorias do dataset")

,Data,Tipo Receita/Despesa,Valor
51,23/01/25,Despesa Combustivel,"R$ 25,00"
37,16/01/25,Despesa Combustivel,"R$ 10,00"
134,05/03/25,Despesa Combustivel,"R$ 25,00"
154,15/03/25,Receita,"R$ 75,00"
17,09/01/25,Receita,"R$ 120,00"


In [7]:
df_dados.tail().style.set_caption("Ultimas 5 linhas do dataset")

,Data,Tipo Receita/Despesa,Valor
152,14/03/25,Receita,"R$ 70,00"
153,14/03/25,Despesa Combustivel,"R$ 20,00"
154,15/03/25,Receita,"R$ 75,00"
155,15/03/25,Despesa Combustivel,"R$ 10,00"
156,16/03/25,Despesa Casa,"R$ 150,00"


In [8]:
f"O dataset possui {df_dados.shape[0]} linhas e {df_dados.shape[1]} colunas."

'O dataset possui 157 linhas e 3 colunas.'

In [11]:
for col in df_dados.columns:
    print(f"- {col}")

- Data
- Tipo Receita/Despesa
- Valor


In [14]:
df_dados.dtypes

Data                    object
Tipo Receita/Despesa    object
Valor                   object
dtype: object

In [15]:
print("Informações gerais do dataset:")
df_dados.info()

Informações gerais do dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Data                  157 non-null    object
 1   Tipo Receita/Despesa  157 non-null    object
 2   Valor                 157 non-null    object
dtypes: object(3)
memory usage: 3.8+ KB
